In [101]:
from ollama import chat
from ollama import ChatResponse
import pandas as pd
import re

In [131]:
def llm_classifier(message: str, symptoms: list, batch_few_shot: dict) -> str:
    # response = chat(model="qwen2.5:3b", messages=[
    #     {
    #         'role': 'user',
    #         'content': f'Based on the provided list of symtoms {symptoms}, I want to classify, with one or more of the classes present in the list, the following sentence: \"{message}\", without any words, just a plain list the predictions. Here are some predefined sentences for fine-tuning:{df}'
    #     }
    # ])
    response = chat(model="qwen3:4b", messages=[
    {
        "role": "user",
        "content": (
            f"You are an AI model trained to classify medical symptoms. "
            f"Given a list of possible classes {symptoms} and a new sentence: \"{message}\", "
            f"predict the relevant classes from the list. Only return a JSON array with the predicted class names, "
            f"without any explanation, formatting, or extra text.\n"
            # f"Here are definitions and their labels for context:\n{batch_few_shot}\n"
            f"As a medical AI, return classes that you are very sure about\n\n"
            f"Your answer must be a valid Python list of strings."
        )
    }
])
    return response.message.content

In [103]:
df = pd.read_csv("description_labelling.csv")

In [104]:
symptoms = df['label'].unique().tolist()

In [129]:
message = "I had unprotected sex with unknown person, extra marriage. It hurts when walking, and I have pain my ass. I am fatigued"

In [106]:
batching_examples = {}

In [107]:
for _, row in df.iterrows():
    batching_examples[row[-1]] = row[0]

/var/folders/mj/9j67468n7s90t742b29r6zr80000gn/T/ipykernel_52364/1614798093.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  batching_examples[row[-1]] = row[0]


In [108]:
slice_len = 20
iterations = int(len(batching_examples) / slice_len)

In [109]:
len(batching_examples)

131

In [ ]:
# for i in range(iterations + 1):
#     start = i * slice_len
#     stop = min((i + 1) * slice_len, len(batching_examples))
#     batch = {k: batching_examples[k] for k in list(batching_examples)[start:stop]}
#     print(qwen2_5_classifier(message, list(batch.keys()), batch))


print(llm_classifier(message, symptoms, {}))

<think>
Okay, let's tackle this query. The user provided a sentence and a list of possible classes. I need to classify the symptoms mentioned into the correct classes.

First, the sentence says, "I had unprotected sex with unknown person, extra marriage. It hurts when walking, and I have pain my ass. I am fatigued."

Breaking it down:

1. "Unprotected sex with unknown person" and "extra marriage" – these might relate to sexually transmitted infections (STIs). Looking at the classes, there's "toxic_look_(typhos)" but that's more for typhoid. Wait, maybe "extra_marital_contacts" is a class. Yes, that's listed. So that's one class.

2. "It hurts when walking, and I have pain my ass." – "pain_behind_the_eyes" is a class, but the user is talking about pain in the ass, which is "pain_in_anal_region". Also, "painful_walking" is a class. So those two.

3. "I am fatigued" – the class "fatigue" is present. So that's another one.

Now, checking if there are any other symptoms. The user didn't men